In [40]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import numpy as np
import tensorflow as tf
import cifar10
from cifar10 import num_classes
from keras.preprocessing.image import ImageDataGenerator
cifar10.maybe_download_and_extract()

Data has apparently already been downloaded and unpacked.


In [41]:
train_data,train_labels,onehot_train=cifar10.load_training_data()
test_data,test_labels,onehot_test=cifar10.load_test_data()
train_data=train_data.astype('float32')
train_labels=train_labels.astype('int32')
test_data=test_data.astype('float32')
test_labels=test_labels.astype('int32')
tf.logging.set_verbosity(tf.logging.ERROR)

mean = np.mean(train_data,axis=(0,1,2,3))
std = np.std(train_data,axis=(0,1,2,3))
train_data = (train_data-mean)/(std+1e-7)
test_data = (test_data-mean)/(std+1e-7)

Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_1
<_io.BufferedReader name='data/CIFAR-10/cifar-10-batches-py/data_batch_1'>
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_2
<_io.BufferedReader name='data/CIFAR-10/cifar-10-batches-py/data_batch_2'>
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_3
<_io.BufferedReader name='data/CIFAR-10/cifar-10-batches-py/data_batch_3'>
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_4
<_io.BufferedReader name='data/CIFAR-10/cifar-10-batches-py/data_batch_4'>
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_5
<_io.BufferedReader name='data/CIFAR-10/cifar-10-batches-py/data_batch_5'>
Loading data: data/CIFAR-10/cifar-10-batches-py/test_batch
<_io.BufferedReader name='data/CIFAR-10/cifar-10-batches-py/test_batch'>


In [42]:
weight_decay = 1e-4
regularizer = tf.contrib.layers.l2_regularizer(scale=weight_decay)
filter_size = 3
feature_maps = 32

In [43]:
print(train_data.shape)

(50000, 32, 32, 3)


In [44]:
datagen = ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=False
    )
datagen.fit(train_data)

In [45]:
print(type(datagen))
a = datagen.flow(train_data,train_labels, batch_size=55000)
x,y = a.next()
print(x.shape,y.shape,type(x),type(y))

<class 'keras.preprocessing.image.ImageDataGenerator'>
(50000, 32, 32, 3) (50000,) <class 'numpy.ndarray'> <class 'numpy.ndarray'>


In [46]:
train_data = x
train_labels = y

In [47]:
def cnn_model_fn(features, labels, mode):
      input_layer = tf.reshape(features["x"], [-1, 32, 32,3])
      conv1 = tf.contrib.layers.conv2d(
          inputs = input_layer,
          num_outputs = feature_maps,
          kernel_size = filter_size,
          weights_regularizer=regularizer,
          )
      
      conv1_bn =  tf.contrib.layers.batch_norm(
            inputs=conv1,
            fused = True
      )
        
      conv2 = tf.contrib.layers.conv2d(
          inputs = conv1_bn,
          num_outputs = feature_maps,
          kernel_size = filter_size,
          weights_regularizer=regularizer,
          )

      conv2_bn =  tf.contrib.layers.batch_norm(
            inputs=conv2,
            fused = True
        )
      

      pool1 =tf.contrib.layers.max_pool2d(inputs = conv2_bn,kernel_size = [2,2],stride = 2)
#       print ("********************************************\n")
#       print(pool1.shape)

      #dropout1 = tf.layers.dropout(inputs=pool1, rate=0.5, training=mode == tf.estimator.ModeKeys.TRAIN)
      dropout1 = tf.contrib.layers.dropout(inputs=pool1, keep_prob=0.8)

      conv3 = tf.contrib.layers.conv2d(
          inputs = dropout1,
          num_outputs = 2*feature_maps,
          kernel_size = filter_size,
          weights_regularizer=regularizer,
          )

      conv3_bn =  tf.contrib.layers.batch_norm(
            inputs=conv3,
            fused = True
        )

      conv4 = tf.contrib.layers.conv2d(
          inputs = conv3_bn,
          num_outputs = 2*feature_maps,
          kernel_size = filter_size,
          weights_regularizer=regularizer,
          )

      conv4_bn =  tf.contrib.layers.batch_norm(
            inputs=conv4,
            fused = True
        )

      
      pool2 = tf.contrib.layers.max_pool2d(inputs = conv4_bn,kernel_size = [2,2],stride = 2)
    
      dropout2 =  tf.contrib.layers.dropout(inputs=pool2, keep_prob=0.7)

      conv5 = tf.contrib.layers.conv2d(
          inputs = dropout2,
          num_outputs = 4*feature_maps,
          kernel_size = filter_size,
          weights_regularizer=regularizer,
          )


      conv5_bn =  tf.contrib.layers.batch_norm(
            inputs=conv5,
            fused = True
        )

      conv6 = tf.contrib.layers.conv2d(
          inputs = conv5_bn,
          num_outputs = 4*feature_maps,
          kernel_size = filter_size,
          weights_regularizer=regularizer,
          )

      conv6_bn =  tf.contrib.layers.batch_norm(
            inputs=conv6,
            fused = True
        )

      
      pool3 = tf.contrib.layers.max_pool2d(inputs = conv6_bn,kernel_size = [2,2] ,stride = 2)
      dropout3 =  tf.contrib.layers.dropout(inputs=pool3, keep_prob=0.6)
      

      pool3_flat = tf.reshape(dropout3, [-1, 4 * 4 * 128])
    
      logits = tf.layers.dense(inputs=pool3_flat, units=10)
      predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.contrib.layers.softmax(logits)
  }
      if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

      # Calculate Loss (for both TRAIN and EVAL modes)
      loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

      # Configure the Training Op (for TRAIN mode)
      if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

      # Add evaluation metrics (for EVAL mode)
      eval_metric_ops = {
          "accuracy": tf.metrics.accuracy(
              labels=labels, predictions=predictions["classes"]),
      }
      return tf.estimator.EstimatorSpec(
          mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)


In [48]:
def current_loss(flag):
    train_eval_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": train_data},
      y=train_labels,
      shuffle=False)
    eval_result1=cifar_classifier.evaluate(input_fn=train_eval_input_fn)
    if flag:
        print("%%%%%%% Train accuracy %%%%%%%%%%%%\n",eval_result1)
    [a,b,c] = eval_result1.items()

    # Evaluate the model and print results
    eval_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": test_data},
      y=test_labels,
      num_epochs=1,
      shuffle=False)
    eval_result2=cifar_classifier.evaluate(input_fn=eval_input_fn)
    if flag:
        print("######### Test accuracy #############\n",eval_result2)
    [d,e,f] = eval_result2.items()
    return a[1],d[1]

In [49]:
# Create the Estimator
cifar_classifier = tf.estimator.Estimator(
  model_fn=cnn_model_fn, model_dir="./cifar_SOTA")


# Set up logging for predictions
# Log the values in the "Softmax" tensor with label "probabilities"
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(
  tensors=tensors_to_log, every_n_iter=50)
print("started\n")
# Train the model
r = 140
for i in range (1,r):
    train_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": train_data},
      y=train_labels,
      batch_size=128,
      num_epochs=None,
      shuffle=True)

    cifar_classifier.train(
      input_fn=train_input_fn,
      steps= 1000,
      hooks=None)
    curr_result = current_loss(1)
#     if curr_result[0] - curr_result[1] > 0.10 or curr_result[1] > 0.98:
#        break
    #print(curr_result)
print ("*****************END*******************")

started

%%%%%%% Train accuracy %%%%%%%%%%%%
 {'accuracy': 0.61058, 'loss': 1.2245682, 'global_step': 1000}
######### Test accuracy #############
 {'accuracy': 0.6292, 'loss': 1.2465457, 'global_step': 1000}
%%%%%%% Train accuracy %%%%%%%%%%%%
 {'accuracy': 0.67944, 'loss': 1.00499, 'global_step': 2000}
######### Test accuracy #############
 {'accuracy': 0.6756, 'loss': 1.0438511, 'global_step': 2000}
%%%%%%% Train accuracy %%%%%%%%%%%%
 {'accuracy': 0.73272, 'loss': 0.8055807, 'global_step': 3000}
######### Test accuracy #############
 {'accuracy': 0.724, 'loss': 0.8578722, 'global_step': 3000}
%%%%%%% Train accuracy %%%%%%%%%%%%
 {'accuracy': 0.7634, 'loss': 0.7067785, 'global_step': 4000}
######### Test accuracy #############
 {'accuracy': 0.7441, 'loss': 0.7673799, 'global_step': 4000}
%%%%%%% Train accuracy %%%%%%%%%%%%
 {'accuracy': 0.78688, 'loss': 0.6501443, 'global_step': 5000}
######### Test accuracy #############
 {'accuracy': 0.763, 'loss': 0.7658761, 'global_step': 5000}
%

%%%%%%% Train accuracy %%%%%%%%%%%%
 {'accuracy': 0.94126, 'loss': 0.1730158, 'global_step': 42000}
######### Test accuracy #############
 {'accuracy': 0.8089, 'loss': 0.7927156, 'global_step': 42000}
%%%%%%% Train accuracy %%%%%%%%%%%%
 {'accuracy': 0.9372, 'loss': 0.18375523, 'global_step': 43000}
######### Test accuracy #############
 {'accuracy': 0.8061, 'loss': 0.85918224, 'global_step': 43000}
%%%%%%% Train accuracy %%%%%%%%%%%%
 {'accuracy': 0.93712, 'loss': 0.18218586, 'global_step': 44000}
######### Test accuracy #############
 {'accuracy': 0.8065, 'loss': 0.7762506, 'global_step': 44000}
%%%%%%% Train accuracy %%%%%%%%%%%%
 {'accuracy': 0.93722, 'loss': 0.19272833, 'global_step': 45000}
######### Test accuracy #############
 {'accuracy': 0.808, 'loss': 0.86803806, 'global_step': 45000}
%%%%%%% Train accuracy %%%%%%%%%%%%
 {'accuracy': 0.9371, 'loss': 0.19503143, 'global_step': 46000}
######### Test accuracy #############
 {'accuracy': 0.8098, 'loss': 0.8381533, 'global_step':

%%%%%%% Train accuracy %%%%%%%%%%%%
 {'accuracy': 0.95478, 'loss': 0.1319738, 'global_step': 83000}
######### Test accuracy #############
 {'accuracy': 0.8157, 'loss': 0.8702414, 'global_step': 83000}
%%%%%%% Train accuracy %%%%%%%%%%%%
 {'accuracy': 0.95848, 'loss': 0.13327138, 'global_step': 84000}
######### Test accuracy #############
 {'accuracy': 0.8068, 'loss': 0.9254466, 'global_step': 84000}
%%%%%%% Train accuracy %%%%%%%%%%%%
 {'accuracy': 0.95462, 'loss': 0.1310219, 'global_step': 85000}
######### Test accuracy #############
 {'accuracy': 0.8123, 'loss': 1.134902, 'global_step': 85000}
%%%%%%% Train accuracy %%%%%%%%%%%%
 {'accuracy': 0.95788, 'loss': 0.12462614, 'global_step': 86000}
######### Test accuracy #############
 {'accuracy': 0.8074, 'loss': 0.93799466, 'global_step': 86000}
%%%%%%% Train accuracy %%%%%%%%%%%%
 {'accuracy': 0.95598, 'loss': 0.13674329, 'global_step': 87000}
######### Test accuracy #############
 {'accuracy': 0.8099, 'loss': 0.94463164, 'global_step'

######### Test accuracy #############
 {'accuracy': 0.813, 'loss': 1.0202974, 'global_step': 123000}
%%%%%%% Train accuracy %%%%%%%%%%%%
 {'accuracy': 0.9634, 'loss': 0.10540908, 'global_step': 124000}
######### Test accuracy #############
 {'accuracy': 0.8187, 'loss': 1.0099766, 'global_step': 124000}
%%%%%%% Train accuracy %%%%%%%%%%%%
 {'accuracy': 0.96344, 'loss': 0.11572289, 'global_step': 125000}
######### Test accuracy #############
 {'accuracy': 0.8191, 'loss': 0.92608094, 'global_step': 125000}
%%%%%%% Train accuracy %%%%%%%%%%%%
 {'accuracy': 0.96318, 'loss': 0.11263033, 'global_step': 126000}
######### Test accuracy #############
 {'accuracy': 0.815, 'loss': 1.068405, 'global_step': 126000}
%%%%%%% Train accuracy %%%%%%%%%%%%
 {'accuracy': 0.96194, 'loss': 0.16073723, 'global_step': 127000}
######### Test accuracy #############
 {'accuracy': 0.8164, 'loss': 1.0410898, 'global_step': 127000}
%%%%%%% Train accuracy %%%%%%%%%%%%
 {'accuracy': 0.96798, 'loss': 0.10944632, 'globa

In [50]:
current_loss(0)

(0.95786, 0.813)